In [1]:
!pip install transformers

In [2]:
#importing the pretrained biobert tokenizer and the biobert model
from transformers import AutoTokenizer, TFAutoModel

# Load the standard BERT tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the standard BERT model
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [3]:
#displaying biobert model summary
bert_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
!pip install datasets

In [5]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("toughdata/quora-question-answer-dataset", split='train')

# Access the question and answer columns
questions = dataset['question']
answers = dataset['answer']

# Combine questions and answers into a list of dictionaries (if desired)
qadata = [{"question": q, "answer": a} for q, a in zip(questions, answers)]

# Print a sample to verify
print(qadata[0])


{'question': 'Why whenever I get in the shower my girlfriend want to join?', 'answer': 'Isn’t it awful? You would swear that there wasn’t enough hot water to go around!\n'}


In [6]:
#function to extract question,answer and tags from the json objects
def extract_answer_question_tags(json_data):
  questions=[]
  answers=[]

  for i in json_data:
    questions.append(i['question'])
    answers.append(i['answer'])

  return questions,answers
#extracting the question,answer and tags from the medical json objects
all_questions=[]
all_answers=[]

for i in [qadata]:
  questions,answers=extract_answer_question_tags(i)
  all_questions.extend(questions)
  all_answers.extend(answers)

print(len(all_questions),len(all_answers))

56402 56402


In [7]:
#loading and storing the medical data into dataframe and csv file correspondingly.
import pandas as pd
all_data=pd.DataFrame({'questions':all_questions,'answers':all_answers})
all_data.to_csv('all_data.csv',index=False)

In [8]:
#loading medical data from the disk
all_data=pd.read_csv('all_data.csv')

In [9]:
all_data

,questions,answers
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...
...,...,...
56397,"Alexandria Ocasio-Cortez said ""Going by track ...","I think she’s right, one is a homosexual with ..."
56398,Is becoming a doctor financially worth it?,Yes if you want to help people and eliminate p...
56399,Where can one find the best biryani in bangalore?,Biryani crafts.These guys will give proper aut...
56400,Which smartphone is best for middle class people?,Oneplus nord\n[LINKED_TEXT: https://latesttech...


In [10]:
#displaying the data
all_data.head()

,questions,answers
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...


In [11]:
import re
#preprocessing questions and answers.
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase


def preprocess(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'

    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


all_data['preprocessed_question'] = all_data['questions'].apply(preprocess)
all_data['preprocessed_answer'] = all_data['answers'].apply(preprocess)
all_data.head()

,questions,answers,preprocessed_question,preprocessed_answer
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...,why whenever i get in the shower my girlfriend...,is not it awful you would swear that there was...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...,what is a proxy and how can i use one,a proxy server is a system or router that prov...
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n,what song has the lyrics someone left the cake...,macarthur is park\n
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...,i am the owner of an adult website called http...,do not let apps that are liers put adds on you...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...,does the bible mention anything about a place ...,st john in the book of revelation mentions an ...


In [12]:
#finding the lens of preprocessed questions and answers
all_data['question_len']=all_data['preprocessed_question'].apply(lambda x: len(x.split(' ')))
all_data['answer_len']=all_data['preprocessed_answer'].apply(lambda x: len(x.split(' ')))

In [13]:
#printing different percentiles of question and answer lengths
import numpy as np
for i in range(0,101,10):
  print(i,np.percentile(all_data.question_len,i),np.percentile(all_data.answer_len,i))

0 1.0 1.0
10 7.0 9.0
20 8.0 19.0
30 9.0 33.0
40 10.0 51.0
50 11.0 73.0
60 13.0 104.0
70 15.0 150.0
80 20.0 225.0
90 27.0 364.0
100 55.0 73296.0


In [14]:
#printing different percentiles of question and answer lengths
for i in range(90,101,1):
  print(i,np.percentile(all_data.question_len,i),np.percentile(all_data.answer_len,i))

90 27.0 364.0
91 28.0 387.0
92 29.0 412.0
93 30.0 443.0
94 33.0 479.0
95 34.0 524.0
96 37.0 585.0
97 39.970000000001164 663.0
98 42.0 815.0
99 45.0 1106.0
100 55.0 73296.0


In [15]:
#since more than 100% percent of questions and about 99% of all answers lie within 500 words, we use 500 as the max length
all_data['short_question']=all_data.apply(lambda x: ' '.join(x.preprocessed_question.split(' ')[:500]) if x.question_len>60 else x.preprocessed_question ,axis=1)
all_data['short_answer']=all_data.apply(lambda x: ' '.join(x.preprocessed_answer.split(' ')[:500]) if x.answer_len>600 else x.preprocessed_answer ,axis=1)

In [16]:
#displaying the data post truncating the question and answers to length of 500
all_data.head(5)

,questions,answers,preprocessed_question,preprocessed_answer,question_len,answer_len,short_question,short_answer
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...,why whenever i get in the shower my girlfriend...,is not it awful you would swear that there was...,12,17,why whenever i get in the shower my girlfriend...,is not it awful you would swear that there was...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...,what is a proxy and how can i use one,a proxy server is a system or router that prov...,10,291,what is a proxy and how can i use one,a proxy server is a system or router that prov...
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n,what song has the lyrics someone left the cake...,macarthur is park\n,13,3,what song has the lyrics someone left the cake...,macarthur is park\n
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...,i am the owner of an adult website called http...,do not let apps that are liers put adds on you...,24,120,i am the owner of an adult website called http...,do not let apps that are liers put adds on you...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...,does the bible mention anything about a place ...,st john in the book of revelation mentions an ...,12,31,does the bible mention anything about a place ...,st john in the book of revelation mentions an ...


In [17]:
#splitting the data into train and validation
from sklearn.model_selection import train_test_split
train, validation = train_test_split(all_data, test_size=0.2,random_state=42,shuffle=True)

In [18]:
#saving train and validation data to disk
train.to_csv('train_data_chatbot.csv',index=False)
validation.to_csv('validation_data_chatbot.csv',index=False)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
cd 'drive/My Drive'


/content/drive/My Drive


In [21]:
#loading train and validation data from disk
import pandas as pd
train=pd.read_csv('/content/train_data_chatbot.csv')
validation=pd.read_csv('/content/validation_data_chatbot.csv')

In [22]:
#displaying the train dataset
train.head(5)

,questions,answers,preprocessed_question,preprocessed_answer,question_len,answer_len,short_question,short_answer
0,With how hard it is being an undocumented immi...,Why reward their dishonesty and criminal behav...,with how hard it is being an undocumented immi...,why reward their dishonesty and criminal behav...,15,25,with how hard it is being an undocumented immi...,why reward their dishonesty and criminal behav...
1,How much longer can the Russian economy sustai...,"However, the conflict between Russia and Ukrai...",how much longer can the russian economy sustai...,however the conflict between russia and ukrain...,12,283,how much longer can the russian economy sustai...,however the conflict between russia and ukrain...
2,Would it be better to know who New York City g...,I'm pretty sure he/she uses anonymity as both ...,would it be better to know who new york city g...,i am pretty sure heshe uses anonymity as both ...,23,35,would it be better to know who new york city g...,i am pretty sure heshe uses anonymity as both ...
3,Are Hong Kong activists still protesting in 2021?,"No, but the terrorists funded by CIA are. Just...",are hong kong activists still protesting in 2021,no but the terrorists funded by cia are just t...,8,54,are hong kong activists still protesting in 2021,no but the terrorists funded by cia are just t...
4,Could Superman surpass Odin in power with Sundip?,most probably\n,could superman surpass odin in power with sundip,most probably\n,8,2,could superman surpass odin in power with sundip,most probably\n


In [23]:
#extracting the short answers and questions from the train data
questions=train['short_question']
answers=train['short_answer']


In [24]:
# #Setting Max_length
# Q_MAX_LENGTH_ = 60
# A_MAX_LENGTH_ = 600
# import tensorflow as tf


# # Tokenize, filter and pad sentences
# def tokenize_and_filter(questions, answers):
#   tokenized_questions, tokenized_answers = [], []

#   for (question, answer) in zip(questions, answers):
#     # generating sequences
#     tokenized_question =  bert_tokenizer.encode(question)
#     tokenized_answer = bert_tokenizer.encode(answer)
#     tokenized_questions.append(tokenized_question)
#     tokenized_answers.append(tokenized_answer)

#   # padding the sequences
#   tokenized_questions = tf.keras.preprocessing.sequence.pad_sequences(
#       tokenized_questions, maxlen=60,padding='post')
#   tokenized_answers = tf.keras.preprocessing.sequence.pad_sequences(
#       tokenized_answers, maxlen=600,padding='post')

#   return tokenized_questions, tokenized_answers

# #tokenizing and padding the train questions and answers
# questions, answers = tokenize_and_filter(questions, answers)
#Setting Max_length
Q_MAX_LENGTH_ = 60
A_MAX_LENGTH_ = 600
import tensorflow as tf


# Tokenize, filter and pad sentences
def tokenize_and_filter(questions, answers):
  tokenized_questions, tokenized_answers = [], []

  for (question, answer) in zip(questions, answers):
    # generating sequences
    # Convert question and answer to strings before encoding
    tokenized_question =  bert_tokenizer.encode(str(question))
    tokenized_answer = bert_tokenizer.encode(str(answer))
    tokenized_questions.append(tokenized_question)
    tokenized_answers.append(tokenized_answer)

  # padding the sequences
  tokenized_questions = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_questions, maxlen=60,padding='post')
  tokenized_answers = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_answers, maxlen=600,padding='post')

  return tokenized_questions, tokenized_answers

#tokenizing and padding the train questions and answers
questions, answers = tokenize_and_filter(questions, answers)

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [ ]:
#preparing the question mask and the answer mask of the train dataset
train_question_mask=[[1 if token!=0 else 0 for token in question] for question in questions]
train_answer_mask=[[1 if token!=0 else 0 for token in answer] for answer in answers]

In [ ]:
#extracting the short answers and questions from the validation data
val_questions=validation['short_question']
val_answers=validation['short_answer']


In [ ]:
#tokenizing and padding the validation question and answers
val_questions, val_answers = tokenize_and_filter(val_questions, val_answers)

In [ ]:
#preparing the question and answer mask for the validation data
val_question_mask=[[1 if token!=0 else 0 for token in question] for question in val_questions]
val_answer_mask=[[1 if token!=0 else 0 for token in answer] for answer in val_answers]

In [ ]:
#preparing the training  dataset
import tensorflow as tf
BATCH_SIZE = 32
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'question': questions,
        'answer': answers,
        'question_mask':train_question_mask,
        'answer_mask':train_answer_mask
    },

))

# dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#preparing the validation dataset
import tensorflow as tf
BATCH_SIZE = 32
BUFFER_SIZE = 10000
val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'question': val_questions,
        'answer': val_answers,
        'question_mask':val_question_mask,
        'answer_mask':val_answer_mask
    },

))

val_dataset = val_dataset.cache()
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE,drop_remainder=True)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

Upto Training and Validation dataset is done. Preprocessing is also done.

In [25]:
import pandas as pd
from datasets import Dataset

# Example data
data = {
    'question': questions.tolist(),
    'answer': answers.tolist()
}

df = pd.DataFrame(data)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Process the dataset to the required format
def preprocess_data(example):
    return {
        'input_text': "question: " + str(example['question']),
        'target_text': str(example['answer'])
    }

dataset = dataset.map(preprocess_data)

# Split the dataset into train and validation sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [27]:
def tokenize_data(example):
    input_text = example['input_text']
    target_text = example['target_text']

    # Tokenize the inputs and labels
    inputs = tokenizer(input_text, max_length=512, truncation=True, padding='max_length', return_tensors='tf')
    targets = tokenizer(target_text, max_length=512, truncation=True, padding='max_length', return_tensors='tf')

    example['input_ids'] = inputs.input_ids[0]
    example['attention_mask'] = inputs.attention_mask[0]
    example['labels'] = targets.input_ids[0]

    return example

train_dataset = train_dataset.map(tokenize_data, batched=False)
val_dataset = val_dataset.map(tokenize_data, batched=False)


Map:   0%|          | 0/40608 [00:00<?, ? examples/s]

Map:   0%|          | 0/4513 [00:00<?, ? examples/s]

In [28]:
import tensorflow as tf

def convert_to_tf_dataset(hf_dataset):
    return tf.data.Dataset.from_generator(
        lambda: iter(hf_dataset),
        output_signature=(
            {
                'input_ids': tf.TensorSpec(shape=(512,), dtype=tf.int32),
                'attention_mask': tf.TensorSpec(shape=(512,), dtype=tf.int32)
            },
            tf.TensorSpec(shape=(512,), dtype=tf.int32)
        )
    )

tf_train_dataset = convert_to_tf_dataset(train_dataset)
tf_val_dataset = convert_to_tf_dataset(val_dataset)

tf_train_dataset = tf_train_dataset.shuffle(len(train_dataset)).batch(8).repeat(2)
tf_val_dataset = tf_val_dataset.batch(8)


In [29]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def convert_to_torch_dataset(hf_dataset):
    input_ids = torch.tensor([example['input_ids'] for example in hf_dataset])
    attention_mask = torch.tensor([example['attention_mask'] for example in hf_dataset])
    labels = torch.tensor([example['labels'] for example in hf_dataset])

    return TensorDataset(input_ids, attention_mask, labels)

train_torch_dataset = convert_to_torch_dataset(train_dataset)
val_torch_dataset = convert_to_torch_dataset(val_dataset)

train_dataloader = DataLoader(train_torch_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_torch_dataset, batch_size=8)


In [ ]:
# from transformers import AdamW, get_scheduler

# optimizer = AdamW(model.parameters(), lr=3e-5)
# num_epochs = 2
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         input_ids, attention_mask, labels = [x.to(device) for x in batch]
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=3e-5)
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10152 [00:00<?, ?it/s]

In [ ]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric = tf.keras.metrics.SparseCategoricalAccuracy()

# model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# history = model.fit(
#     tf_train_dataset,
#     epochs=2,
#     validation_data=tf_val_dataset
# )


In [ ]:
model.save_pretrained('/content/drive/My Drive/path_to_save_model')
tokenizer.save_pretrained('/content/drive/My Drive/path_to_save_model')


In [ ]:
def predict(question):
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors='tf', max_length=512, truncation=True, padding='max_length')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

# Test the model on a custom input
custom_question = "What is the capital of Italy?"
prediction = predict(custom_question)
print(f'Prediction: {prediction}')


In [ ]:
import numpy as np
import nltk
from datasets import load_metric
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

nltk.download('punkt')

# Load evaluation metrics
bleu = load_metric('bleu')
rouge = load_metric('rouge')
# rouge = load_metric('rouge', cache_dir="./")

def evaluate_model(dataset, tokenizer, model):
    predictions = []
    references = []

    for example in dataset:
        question = example['question']
        reference = example['answer']

        input_text = "question: " + question
        inputs = tokenizer(input_text, return_tensors='tf', max_length=512, truncation=True, padding='max_length')
        input_ids = inputs.input_ids
        attention_mask = inputs.attention_mask

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        predictions.append(prediction)
        references.append(reference)

    # Calculate BLEU score
    bleu_score = bleu.compute(predictions=[nltk.word_tokenize(pred) for pred in predictions],
                              references=[[nltk.word_tokenize(ref)] for ref in references])

    # Calculate ROUGE score
    rouge_score = rouge.compute(predictions=predictions, references=references)

    # Calculate F1 score
    def simple_f1(predictions, references):
        preds = [1 if pred == ref else 0 for pred, ref in zip(predictions, references)]
        return f1_score([1] * len(references), preds, average='binary')

    f1 = simple_f1(predictions, references)

    return bleu_score, rouge_score, f1

bleu_score, rouge_score, f1 = evaluate_model(val_dataset, tokenizer, model)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("F1 Score:", f1)


In [ ]:
# Plotting BLEU score
bleu_scores = [bleu_score['bleu']]

plt.figure(figsize=(10, 6))
plt.plot(bleu_scores, marker='o')
plt.title('BLEU Score over Epochs')
plt.xlabel('Epoch')
plt.ylabel('BLEU Score')
plt.show()

# Plotting ROUGE scores
rouge1 = [score['rouge1'].mid.fmeasure for score in rouge_score.values()]
rouge2 = [score['rouge2'].mid.fmeasure for score in rouge_score.values()]
rougeL = [score['rougeL'].mid.fmeasure for score in rouge_score.values()]

plt.figure(figsize=(10, 6))
plt.plot(rouge1, label='ROUGE-1', marker='o')
plt.plot(rouge2, label='ROUGE-2', marker='o')
plt.plot(rougeL, label='ROUGE-L', marker='o')
plt.title('ROUGE Scores over Epochs')
plt.xlabel('Epoch')
plt.ylabel('ROUGE Score')
plt.legend()
plt.show()

# Plotting F1 Score
f1_scores = [f1] # You can modify this if you have multiple scores

plt.figure(figsize=(10, 6))
plt.plot(f1_scores, marker='o')
plt.title('F1 Score over Epochs')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.show()
